In [ ]:
# Load Required Libraries
import tensorflow as tf 
import pandas as pd
import numpy as np
import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers, optimizers, losses, metrics
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# ==========================================Load Data===============================================
(Xtr, Ytr), (Xte, Yte) = mnist.load_data()
print(f"X train Size: {Xtr.shape}\nTraining Target Size: {Ytr.shape}\nX test Size: {Xte.shape}\nTesting Target Size: {Yte.shape}\n")
# ==============================================Plot some digits====================================
fig, ax = plt.subplots(2, 5, figsize = (6, 6))
for i in range(10):
    ax[i//5, i%5].imshow(Xtr[i], cmap='grey')
    ax[i//5, i%5].set_title(Ytr[i])
    ax[i//5, i%5].set_xticks([])
    ax[i//5, i%5].set_yticks([])
plt.tight_layout()
plt.show()
# ===============================================Normalization======================================
scaler = MinMaxScaler()
Xtr_N = scaler.fit_transform(Xtr.reshape(-1, Xtr.shape[1]*Xtr.shape[2]))
Xte_N = scaler.transform(Xte.reshape(-1, Xte.shape[1]*Xte.shape[2]))
print(f"Normalized X_train Size: {Xtr_N.shape}\nNormalized X_test Size: {Xte_N.shape}")
# =====================================================One-hot encoding==============================
Ytr_encoded = to_categorical(Ytr, num_classes=10)
Yte_encoded = to_categorical(Yte, num_classes=10)
# ================================================Model==============================================
model = models.Sequential()
model.add(layers.Dense(100, activation = 'relu', input_shape = (Xtr_N.shape[1],)))
model.add(layers.Dense(50, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))
model.summary()
# =================================================Compile and Train=================================
model.compile(
    loss = losses.CategoricalCrossentropy(),    # SparseCategoricalCrossentropy
    optimizer = optimizers.Adam(),              # Alternatives: RMSprop
    metrics = [metrics.CategoricalAccuracy()] 
)

His = model.fit(Xtr_N, Ytr_encoded, epochs=50, batch_size=32, validation_split=0.2)
df1 = pd.DataFrame(His.history)
df1[['categorical_accuracy', 'val_categorical_accuracy']].plot(kind='line', xlabel='Epochs', ylabel='Accuracy Value',
                                                               title='Accuracy over epochs', legend=True, figsize=(12, 4))
df1[['loss', 'val_loss']].plot(kind='line', xlabel='Epochs', ylabel='Loss Value',
                                                               title='Loss over epochs', legend=True, figsize=(12, 4))
final_test_result = model.evaluate(Xte_N, Yte_encoded)
print(f"Evaluation results:\nVal_loss (%) = {final_test_result[0]*100:.4f}\nVal_accuracy (%) = {final_test_result[1]*100:.4f}")
y_pred = model.predict(Xte_N)   
y_pred = np.argmax(y_pred, axis=1)
y_true_y_pred = np.column_stack((Yte, y_pred))
print(f"\n\nOut of {Yte.shape[0]} testing data, {sum(y_true_y_pred[:, 0] != y_true_y_pred[:, 1])} are incorrect\nabout --> {100*(sum(y_true_y_pred[:, 0] != y_true_y_pred[:, 1])/Yte.shape[0])}%")
